Replicating Idiosyncratic Volatility Result.

In [34]:
#import statements
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from finance_byu.summarize import summary
from finance_byu.regtables import Regtable
from finance_byu.rolling import roll_idio

In [35]:
#load daily CRSP price data
data = pd.read_csv("crsp_daily.csv", parse_dates=['caldt'])
data

,permno,caldt,shrcd,excd,siccd,prc,ret,vol,shr
0,10000,1986-01-07,10,3,3990,-2.5625,NaN,1000.0,3680.0
1,10000,1986-01-08,10,3,3990,-2.5000,-0.024390,12800.0,3680.0
2,10000,1986-01-09,10,3,3990,-2.5000,0.000000,1400.0,3680.0
3,10000,1986-01-10,10,3,3990,-2.5000,0.000000,8500.0,3680.0
4,10000,1986-01-13,10,3,3990,-2.6250,0.050000,5450.0,3680.0
...,...,...,...,...,...,...,...,...,...
105258375,93436,2023-12-22,11,3,9999,252.5400,-0.007701,93148500.0,3178920.0
105258376,93436,2023-12-26,11,3,9999,256.6100,0.016116,86700700.0,3178920.0
105258377,93436,2023-12-27,11,3,9999,261.4400,0.018822,106251000.0,3178920.0
105258378,93436,2023-12-28,11,3,9999,253.1800,-0.031594,113251000.0,3178920.0


In [59]:
df = data.copy()

df['prclag'] = df.groupby('permno')['prc'].shift() #lag price
df['logret'] = np.log(1 + df['ret']) #calculate the logarithm of returns

#query data to only include data specified in paper
df = df.query("'1963-01-01' <= caldt and caldt <= '2000-12-30'").reset_index(drop=True) 

df = df.query("prclag >= 5").reset_index(drop=True) #disregard lost cost stocks

df

,permno,caldt,shrcd,excd,siccd,prc,ret,vol,shr,prclag,logret
0,10001,1986-09-03,11,3,4920,7.0000,0.037037,5350.0,985.0,6.7500,0.036368
1,10001,1986-09-04,11,3,4920,6.7500,-0.035714,7200.0,985.0,7.0000,-0.036367
2,10001,1986-09-05,11,3,4920,6.5000,-0.037037,3940.0,985.0,6.7500,-0.037740
3,10001,1986-09-08,11,3,4920,6.7500,0.054615,1610.0,985.0,6.5000,0.053176
4,10001,1986-09-09,11,3,4920,6.3750,-0.055556,1400.0,985.0,6.7500,-0.057159
...,...,...,...,...,...,...,...,...,...,...,...
30950690,93316,2000-03-10,11,3,5060,15.2500,0.008264,8555.0,3082.0,15.1250,0.008230
30950691,93316,2000-03-13,11,3,5060,15.3750,0.008197,10685.0,3082.0,15.2500,0.008164
30950692,93316,2000-03-14,11,3,5060,15.0625,-0.020325,1800.0,3082.0,15.3750,-0.020534
30950693,93316,2000-03-15,11,3,5060,15.0625,0.000000,2100.0,3082.0,15.0625,0.000000


In [60]:
#import daily market returns, both equal and value weighted
market_returns = pd.read_csv("dsi.csv",parse_dates=['caldt'])
market_returns

,caldt,mkt,ewmkt
0,1926-01-02,0.005689,0.009516
1,1926-01-04,0.000706,0.005780
2,1926-01-05,-0.004821,-0.001927
3,1926-01-06,-0.000423,0.001182
4,1926-01-07,0.004988,0.008453
...,...,...,...
25793,2023-12-22,0.002440,0.007676
25794,2023-12-26,0.005218,0.009674
25795,2023-12-27,0.001995,0.005276
25796,2023-12-28,-0.000108,0.002042


In [61]:
#merge market returns to overall stock dataframe
df = pd.merge(df, market_returns, on = ['caldt'], how = 'left')
df


,permno,caldt,shrcd,excd,siccd,prc,ret,vol,shr,prclag,logret,mkt,ewmkt
0,10001,1986-09-03,11,3,4920,7.0000,0.037037,5350.0,985.0,6.7500,0.036368,0.002953,-0.001926
1,10001,1986-09-04,11,3,4920,6.7500,-0.035714,7200.0,985.0,7.0000,-0.036367,0.012870,0.007118
2,10001,1986-09-05,11,3,4920,6.5000,-0.037037,3940.0,985.0,6.7500,-0.037740,-0.011584,-0.002484
3,10001,1986-09-08,11,3,4920,6.7500,0.054615,1610.0,985.0,6.5000,0.053176,-0.010801,-0.008489
4,10001,1986-09-09,11,3,4920,6.3750,-0.055556,1400.0,985.0,6.7500,-0.057159,-0.002734,-0.004689
...,...,...,...,...,...,...,...,...,...,...,...,...,...
30950690,93316,2000-03-10,11,3,5060,15.2500,0.008264,8555.0,3082.0,15.1250,0.008230,-0.002837,0.000308
30950691,93316,2000-03-13,11,3,5060,15.3750,0.008197,10685.0,3082.0,15.2500,0.008164,-0.014556,-0.008059
30950692,93316,2000-03-14,11,3,5060,15.0625,-0.020325,1800.0,3082.0,15.3750,-0.020534,-0.022405,-0.012365
30950693,93316,2000-03-15,11,3,5060,15.0625,0.000000,2100.0,3082.0,15.0625,0.000000,0.006799,-0.005430


In [65]:
#should we use log ret? also how do we get excess market return? ddof = 0?

df['idio_vol'] = df.groupby('permno')[['ret', 'mkt']].apply(lambda x: roll_idio(x['ret'], x['mkt'], win = 30, minp = 30, ddof = 0)).reset_index(drop=True)
df = df.query("idio_vol == idio_vol").reset_index(drop=True)
df

,permno,caldt,shrcd,excd,siccd,prc,ret,vol,shr,prclag,logret,mkt,ewmkt,idio_vol
0,10001,1986-10-17,11,3,4920,-6.5625,-0.027778,0.0,991.0,6.7500,-0.028171,-0.002519,-0.000335,0.031562
1,10001,1986-10-21,11,3,4920,6.7500,0.058824,100.0,991.0,6.3750,0.057159,-0.000263,-0.000121,0.032630
2,10001,1986-10-22,11,3,4920,7.0000,0.037037,100.0,991.0,6.7500,0.036368,0.001406,0.000868,0.032581
3,10001,1986-10-23,11,3,4920,7.0000,0.000000,2500.0,991.0,7.0000,0.000000,0.010694,0.004210,0.031726
4,10001,1986-10-24,11,3,4920,6.7500,-0.035714,6600.0,991.0,7.0000,-0.036367,-0.002450,0.001582,0.031073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30253556,93316,2000-03-10,11,3,5060,15.2500,0.008264,8555.0,3082.0,15.1250,0.008230,-0.002837,0.000308,0.008796
30253557,93316,2000-03-13,11,3,5060,15.3750,0.008197,10685.0,3082.0,15.2500,0.008164,-0.014556,-0.008059,0.008045
30253558,93316,2000-03-14,11,3,5060,15.0625,-0.020325,1800.0,3082.0,15.3750,-0.020534,-0.022405,-0.012365,0.008413
30253559,93316,2000-03-15,11,3,5060,15.0625,0.000000,2100.0,3082.0,15.0625,0.000000,0.006799,-0.005430,0.008282


In [ ]:
factors = pd.read_csv('factors.csv',parse_dates=['caldt'])
factors

,caldt,exmkt,smb,hml,umd,rf
0,1927-01-31,-0.06,-0.37,4.54,0.36,0.25
1,1927-02-28,4.18,0.04,2.94,-2.14,0.26
2,1927-03-31,0.13,-1.65,-2.61,3.61,0.30
3,1927-04-30,0.46,0.30,0.81,4.30,0.25
4,1927-05-31,5.44,1.53,4.73,3.00,0.30
...,...,...,...,...,...,...
1156,2023-05-31,0.35,1.61,-7.72,-0.63,0.36
1157,2023-06-30,6.46,1.54,-0.26,-2.37,0.40
1158,2023-07-31,3.21,2.08,4.11,-3.98,0.45
1159,2023-08-31,-2.39,-3.16,-1.06,3.77,0.45


In [ ]:
df = df[['permno', 'caldt', 'idio_vol']]
df = pd.merge(df, factors, on = ['caldt'], how = 'inner')
df

,permno,caldt,idio_vol,exmkt,smb,hml,umd,rf
0,10001,1986-10-31,0.033742,4.66,-2.50,-1.42,4.46,0.46
1,10001,1986-11-28,0.036902,1.17,-1.91,-0.07,-0.26,0.39
2,10001,1986-12-31,0.040145,-3.27,0.13,0.36,0.35,0.49
3,10001,1987-01-30,0.042319,12.47,-1.81,-3.16,2.20,0.42
4,10001,1987-02-27,0.039738,4.39,3.49,-5.91,-2.07,0.43
...,...,...,...,...,...,...,...,...
1438476,93316,1999-10-29,0.035129,6.12,-6.64,-3.37,5.50,0.39
1438477,93316,1999-11-30,0.046550,3.37,7.21,-6.12,5.64,0.36
1438478,93316,1999-12-31,0.056021,7.72,7.03,-8.33,13.22,0.44
1438479,93316,2000-01-31,0.055977,-4.74,5.77,-1.88,1.92,0.41


In [ ]:
market_cap = pd.read_csv("market_cap.csv", parse_dates=['caldt'])
market_cap['melag'] = market_cap.groupby('permno')['me'].shift()
market_cap = market_cap.dropna()
df = pd.merge(df, market_cap, on = ['caldt', 'permno'], how = 'left')
df
# market_cap


,permno,caldt,idio_vol,exmkt,smb,hml,umd,rf,ticker,prc,me,ret,shr,melag
0,10001,1986-10-31,0.033742,4.66,-2.50,-1.42,4.46,0.46,GFGC,6.6250,6.56538,0.039216,991.0,6.31763
1,10001,1986-11-28,0.036902,1.17,-1.91,-0.07,-0.26,0.39,GFGC,7.0000,6.93700,0.056604,991.0,6.56538
2,10001,1986-12-31,0.040145,-3.27,0.13,0.36,0.35,0.49,GFGC,7.0000,6.93700,0.015000,991.0,6.93700
3,10001,1987-01-30,0.042319,12.47,-1.81,-3.16,2.20,0.42,GFGC,6.7500,6.68925,-0.035714,991.0,6.93700
4,10001,1987-02-27,0.039738,4.39,3.49,-5.91,-2.07,0.43,GFGC,6.2500,6.19375,-0.074074,991.0,6.68925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1438476,93316,1999-10-29,0.035129,6.12,-6.64,-3.37,5.50,0.39,ZING,6.8750,21.18870,-0.035088,3082.0,21.95930
1438477,93316,1999-11-30,0.046550,3.37,7.21,-6.12,5.64,0.36,ZING,8.5000,26.19700,0.236364,3082.0,21.18870
1438478,93316,1999-12-31,0.056021,7.72,7.03,-8.33,13.22,0.44,ZING,12.8750,39.68080,0.514706,3082.0,26.19700
1438479,93316,2000-01-31,0.055977,-4.74,5.77,-1.88,1.92,0.41,ZING,15.1250,46.61530,0.174757,3082.0,39.68080


In [ ]:
df['bins'] = df.groupby('caldt')['idio_vol'].transform(pd.qcut,5,labels=False)
mcapsum = df.groupby(['caldt','bins'])['melag'].sum()

df['rme'] = df['ret']*df['melag']
port = df.groupby(['caldt','bins'])['rme'].sum() / mcapsum
port = port.unstack(level='bins').rename('p{:.0f}'.format,axis='columns')*100

port

bins,p0,p1,p2,p3,p4
caldt,,,,,
1963-02-28,-2.155624,-2.482508,-2.370972,-0.894379,-1.789163
1963-03-29,3.768627,2.809132,2.428202,2.580721,1.380988
1963-04-30,4.460670,5.031991,6.134559,6.141866,4.072625
1963-05-31,1.440427,2.163238,3.108314,3.758542,8.568521
1963-06-28,-2.035070,-1.272704,-1.961031,-1.178443,1.173851
...,...,...,...,...,...
2000-08-31,5.751909,6.861940,6.207431,12.831400,8.372766
2000-09-29,1.093945,0.314516,-6.143502,-8.560725,-22.352761
2000-10-31,1.393076,3.138987,-0.490207,-4.880098,-12.368185


In [63]:
port['spread'] = port['p4'] - port['p0']
summary(port).loc[['count','mean','std','tstat','pval']].round(3)

bins,p0,p1,p2,p3,p4,spread
count,455.000,455.000,455.000,455.000,455.000,455.000
mean,0.950,1.157,1.327,1.460,2.191,1.241
std,3.729,4.604,5.558,7.108,9.582,8.158
tstat,5.433,5.360,5.094,4.381,4.878,3.246
pval,0.000,0.000,0.000,0.000,0.000,0.001
